In [1]:
import os,sys
import numpy as np
import xarray as xr
import zarr
# from dask.distributed import Client
import multiprocessing.popen_spawn_posix
from distributed import Client

c = Client()
c

Client Scheduler: tcp://127.0.0.1:42345 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 322.12 GB


In [2]:
dir_sc = os.environ['SCRATCH']
dir_work = os.environ['WORK']

In [3]:
prefix = "eORCA025.L75-IMHOTEP"
chunk_size = {"x":133,"y":320}

# ---------------------------------- weight for the GMSL calculation -----------------------------------------------
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(diri+'mesh_hgr.nc',chunks=chunk_size).squeeze()
tmask = mesh_hgr.tmask

e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

# nav_lon =  mesh_hgr.nav_lon
# nav_lat =  mesh_hgr.nav_lat

# constructing the weight
bt = (e1t * e2t).load()# area of each cell
# getting the weights to be applied to every grid cell
# masking the land cells
oceanArea = (bt*tmask[0])  #tmask : 1 on ocean, 0 on land
totalOceanArea = oceanArea.sum()
Weight = (oceanArea/totalOceanArea).compute().drop_vars('nav_lev').drop_vars('time_counter')


In [6]:
%%time
for nexp in ["ES","EAI", "EGAI"]:
    print(nexp)
    # load sea ice mask to apply
    SIMSK_E = xr.open_zarr(dir_work+"/ETUDE3/ENS_VAR_BUDGET/raw_data/"+nexp+"/"+nexp+"_SIMASK15_E1980_2018.zarr").SIMASK15_E #  Sea ice mask
    sum_simsk = SIMSK_E.sum(('ens','time_counter')).compute()
    si_mask = (sum_simsk.where(sum_simsk==390,0)/390)
    
    for COMP in ['STERIC','THERMO', 'HALO']:
        print(COMP)

        for dep in ["50m", "100m", "300m", "700m", "2000m", "bottom"]: 
            print(dep)
            dir_data = dir_sc + "/NEW_ENS_VAR_BUDGET/raw_STERIC_THERMO_HALO/"+nexp+"/"+COMP+"/"+dep+"/"

            sosteric = xr.open_zarr(dir_data+nexp+"1y_steric_0_"+dep+"_1980_2018.zarr").sosteric * si_mask
            print("step GMSL")

            # calculating the GMSL
            mean_E_GMSL = ((sosteric * Weight).sum(('x','y')))
            STERIC_E_corr = sosteric  - mean_E_GMSL
            
            # retrieving the ensemble mean time mean at each point
            print("step tmean")

            tmean_E = STERIC_E_corr.mean('year').mean('ens')
            
            STERIC_E_corr_shifted = ( STERIC_E_corr - tmean_E ).compute()
    
            # save in scratch
            print("step saving")

            diro = dir_sc + "/NEW_ENS_VAR_BUDGET/"+nexp+"/"+COMP+"/"
            STERIC_E_corr_shifted.to_dataset(name=COMP+"_E").to_zarr(diro+COMP+"_0_"+dep+"_2use_1980_2018.zarr", mode='w')

ES
STERIC
50m
step GMSL
step tmean


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


step saving
100m
step GMSL
step tmean


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

step saving
300m
step GMSL
step tmean


...

step saving
2000m
step GMSL
step tmean


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

step saving
bottom
step GMSL
step tmean


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


step saving
CPU times: user 1h 21min 26s, sys: 9min 53s, total: 1h 31min 19s
Wall time: 1h 40min 10s
